In [1]:
#google driveとの連携
from google.colab import drive
drive.mount('/content/drive')

#カレントディレクトリの設定
dir_path = '/content/drive/My Drive/Colab Notebooks/repo8'
import os
os.chdir(dir_path)

#モジュール追加用pathの設定
import sys
sys.path.append('..')

#モジュールのインポート
import numpy as np

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from common import config
config.GPU = True
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
from dataset import ptb
from ch06.better_rnnlm import  BetterRnnlm

In [5]:
# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_data, _, _ = ptb.load_data('test')

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

Done
Done
Done


In [6]:
# モデルの作成
model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

In [7]:
# 訓練
best_ppl = float('inf')
for epoch in range(max_epoch):
  trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size, time_size=time_size, max_grad=max_grad)

  model.reset_state()
  ppl = eval_perplexity(model, corpus_val)
  print('valid perplexity: ', ppl)

  if best_ppl > ppl:
    best_ppl = ppl
    model.save_params()
  else:
    lr /= 4.0
    optimizer.lr = lr
  
  model.reset_state()
  print('-' * 50)

| epoch 1 |  iter 1 / 1327 | time 14[s] | perplexity 10000.00
| epoch 1 |  iter 21 / 1327 | time 20[s] | perplexity 3375.81
| epoch 1 |  iter 41 / 1327 | time 25[s] | perplexity 2110.85
| epoch 1 |  iter 61 / 1327 | time 31[s] | perplexity 1241.21
| epoch 1 |  iter 81 / 1327 | time 36[s] | perplexity 1034.52
| epoch 1 |  iter 101 / 1327 | time 42[s] | perplexity 869.80
| epoch 1 |  iter 121 / 1327 | time 48[s] | perplexity 783.23
| epoch 1 |  iter 141 / 1327 | time 53[s] | perplexity 718.77
| epoch 1 |  iter 161 / 1327 | time 59[s] | perplexity 702.11
| epoch 1 |  iter 181 / 1327 | time 65[s] | perplexity 679.12
| epoch 1 |  iter 201 / 1327 | time 70[s] | perplexity 600.49
| epoch 1 |  iter 221 / 1327 | time 76[s] | perplexity 574.83
| epoch 1 |  iter 241 / 1327 | time 82[s] | perplexity 526.04
| epoch 1 |  iter 261 / 1327 | time 87[s] | perplexity 552.87
| epoch 1 |  iter 281 / 1327 | time 93[s] | perplexity 514.63
| epoch 1 |  iter 301 / 1327 | time 99[s] | perplexity 460.85
| epoch 

最終的なパープレキシティが大きく減っていることが確認できた。

**参考文献**
1. ゼロから作る Deep Learning 2 自然言語処理偏